# GA Data Science 18 (DAT18) - Lab4

## Lab goals

- Simple APIs with Python
- Kimono Labs: an external tool for aquiring data

In [2]:
# Imports at the top
import json
import urllib
import pandas as pd

import requests  ## What we use to read from a URL -- when you read a webpage?
import json


## Using Python APIs

### Either Use Sheetsu or an API you've found
Sheetsu: Turning Google Docs into APIs/
https://sheetsu.com/

### Using Sheetsu
I've created a GoogleDoc with wine data

https://docs.google.com/spreadsheets/d/1_U-DHIrBXN8A1e3NbyOIJRORge2udmmaa8qS9mKddWo/edit#gid=0 
https://sheetsu.com/apis/79656330

In [3]:
# You can either post or get info from this API
# using the /column/Name you can pull only specfic subsets of the data
api_base_url = 'https://sheetsu.com/apis/79656330'
api_column = '/column/Name'  ## Name is a column in the spreadsheet, but it could be called anything


##### Reading Data from an API datafeed

In [4]:
# What kind of data is this returning?
api_response = requests.get(api_base_url)
api_response.text

u'{"status":200,"success":true,"result":[{"Color":"W","Region":"Portugal","Country":"Portugal","Vintage":"2013","Vinyard":"Vinho Verde","Name":"","Grape":"","Consumed In":"2015","Score":"4","Price":""},{"Color":"W","Region":"France","Country":"France","Vintage":"2013","Vinyard":"Peyruchet","Name":"","Grape":"","Consumed In":"2015","Score":"3","Price":"17.8"},{"Color":"W","Region":"Oregon","Country":"Oregon","Vintage":"2013","Vinyard":"Abacela","Name":"","Grape":"","Consumed In":"2015","Score":"3","Price":"20"},{"Color":"W","Region":"Spain","Country":"Spain","Vintage":"2012","Vinyard":"Ochoa","Name":"","Grape":"chardonay","Consumed In":"2015","Score":"2.5","Price":"7"},{"Color":"R","Region":"","Country":"US","Vintage":"2012","Vinyard":"Heartland","Name":"Spice Trader","Grape":"chiraz, cab","Consumed In":"2015","Score":"3","Price":"6"},{"Color":"R","Region":"California","Country":"US","Vintage":"2012","Vinyard":"Crow Canyon","Name":"","Grape":"cab","Consumed In":"2015","Score":"3.5","Pri

In [5]:
# What kind of data is this returning?  JSON
api_column_url = api_base_url + api_column
column_response = requests.get(api_column_url)
column_response.text

u'{"status":200,"success":true,"result":["","","","","Spice Trader","","#14","","","","Rosso Dei Poggi","Montepulciano D\'Abruzzo","","Rosso Di Montalcino","Sancerre Cuvee Des Moulins Bales","Meiomi","Saint Drezery Prestige","Saint Drezery Prestige","Sauvignon Blanc","Uppercut North Coast","Nero d\'Avola","Cape Blush","Pinot Grigio","Cru Classe","Domaine du Claouset","Sauvignon Blanc","Vinho Verde","Vincent Dampt","Albarino","Dylans Wine","Dylans Wine","Dylans Wine","Dylans Wine","Dylans Wine","Dylans Wine","Dylans Wine"]}'

#### Reading JSON

In [6]:
reponse_dict = json.loads(api_response.text)  ## Makes the json file into a dictionary with a large list in it 
## that contains dictionaries

#### What is the response of an API and what does it mean??

In [7]:
# What data was returned?  A dictionary
reponse_dict.keys()

[u'status', u'result', u'success']

In [8]:
reponse_dict['status']

200

#### Lets read the data into a DataFrame!

In [9]:
wine_df = pd.DataFrame(reponse_dict['result'])
wine_df.head()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet
2,W,2015,Oregon,,,20,Oregon,3,2013,Abacela
3,W,2015,Spain,chardonay,,7,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6,,3,2012,Heartland


In [10]:
wine_df.json  ## What do I need to add here?  

AttributeError: 'DataFrame' object has no attribute 'json'

#### Pandas has great functions. We could just do it this way

This sometimes works, but the data may need adjusting

In [11]:
pd.read_json(api_response.text).head(2)

,result,status,success
0,"{u'Grape': u'', u'Name': u'', u'Color': u'W', ...",200,True
1,"{u'Grape': u'', u'Name': u'', u'Color': u'W', ...",200,True


In [12]:
wine_df = pd.DataFrame(list(pd.read_json(api_response.text).result.values))
wine_df.head(2)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet


In [13]:
wine_df.to_json()  ## The default is by columns

'{"Color":{"0":"W","1":"W","2":"W","3":"W","4":"R","5":"R","6":"R","7":"R","8":"R","9":"R","10":"R","11":"R","12":"R","13":"R","14":"W","15":"R","16":"R","17":"R","18":"W","19":"W","20":"R","21":"P","22":"W","23":"R","24":"W","25":"W","26":"W","27":"W","28":"W","29":"R","30":"R","31":"R","32":"R","33":"R","34":"R"},"Consumed In":{"0":"2015","1":"2015","2":"2015","3":"2015","4":"2015","5":"2015","6":"2015","7":"2015","8":"2015","9":"2015","10":"2015","11":"2015","12":"2015","13":"2015","14":"2013","15":"2013","16":"2013","17":"2013","18":"2014","19":"2014","20":"2014","21":"2014","22":"2014","23":"2014","24":"2015","25":"2015","26":"2015","27":"2015","28":"2015","29":"2015","30":"2015","31":"2015","32":"2015","33":"2015","34":"2015"},"Country":{"0":"Portugal","1":"France","2":"Oregon","3":"Spain","4":"US","5":"US","6":"US","7":"France","8":"France","9":"US","10":"Italy","11":"","12":"US","13":"Italy","14":"France","15":"US","16":"France","17":"France","18":"US","19":"US","20":"Italy","2

In [14]:
wine_df.to_json(orient='records')  ## But you can orient it a few different ways

'[{"Color":"W","Consumed In":"2015","Country":"Portugal","Grape":"","Name":"","Price":"","Region":"Portugal","Score":"4","Vintage":"2013","Vinyard":"Vinho Verde"},{"Color":"W","Consumed In":"2015","Country":"France","Grape":"","Name":"","Price":"17.8","Region":"France","Score":"3","Vintage":"2013","Vinyard":"Peyruchet"},{"Color":"W","Consumed In":"2015","Country":"Oregon","Grape":"","Name":"","Price":"20","Region":"Oregon","Score":"3","Vintage":"2013","Vinyard":"Abacela"},{"Color":"W","Consumed In":"2015","Country":"Spain","Grape":"chardonay","Name":"","Price":"7","Region":"Spain","Score":"2.5","Vintage":"2012","Vinyard":"Ochoa"},{"Color":"R","Consumed In":"2015","Country":"US","Grape":"chiraz, cab","Name":"Spice Trader","Price":"6","Region":"","Score":"3","Vintage":"2012","Vinyard":"Heartland"},{"Color":"R","Consumed In":"2015","Country":"US","Grape":"cab","Name":"","Price":"13","Region":"California","Score":"3.5","Vintage":"2012","Vinyard":"Crow Canyon"},{"Color":"R","Consumed In":"2

In [61]:
wine_df.to_json(orient='split')

'{"columns":["Color","Consumed In","Country","Grape","Name","Price","Region","Score","Vintage","Vinyard"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34],"data":[["W","2015","Portugal","","","","Portugal","4","2013","Vinho Verde"],["W","2015","France","","","17.8","France","3","2013","Peyruchet"],["W","2015","Oregon","","","20","Oregon","3","2013","Abacela"],["W","2015","Spain","chardonay","","7","Spain","2.5","2012","Ochoa"],["R","2015","US","chiraz, cab","Spice Trader","6","","3","2012","Heartland"],["R","2015","US","cab","","13","California","3.5","2012","Crow Canyon"],["R","2015","US","","#14","21","Oregon","2.5","2013","Abacela"],["R","2015","France","merlot, cab","","12","Bordeaux","3.5","2012","David Beaulieu"],["R","2015","France","merlot, cab","","11.99","Medoc","3.5","2011","Chantemerle"],["R","2015","US","merlot","","13","Washington","4","2011","Hyatt"],["R","2015","Italy","sangiovese","Rosso Dei Poggi","12","Tuscany",

In [63]:
wine_df.Color.unique()

array([u'W', u'R', u'P'], dtype=object)

#### Writing to an API using PUT
You will rarely need to do this

In [16]:
# Send data to an API
post_data = {
    'Grape' : ''
    , 'Name' : 'Dylans Wine'
    , 'Color' : 'R'
    , 'Country' : 'US'
    , 'Region' : 'CA'
    , 'Vinyard' : ''
    , 'Score' : '10'
    , 'Consumed In' : '2015'
    , 'Vintage' : '1973'
    , 'Price' : '200'
    }
requests.post(api_base_url, data=post_data)


<Response [201]>

### Exercise 1
    - Either with your own google doc or a different API
    - Extract some data using requests
    - Pull the data into a DataFrame
    - Search For missing data
        - Is there any missing data you can clean?
        - Is there any data you can just remove?
    - Summarize the data 
        - Try using describe, min, max, mean, var
        - What are the unique values and can you do anything with the text?
            Hint: Try using pd.get_dummies
    - Bonus: Create some visualizations form this data
    - Bonus: Try using sheetsu to push new data to the google doc (requests.post)

In [55]:
import numpy as np

In [56]:
api_base_url_plomo = 'https://sheetsu.com/apis/36945b1c'
api_column_plomo = '/column/Customer'
api_column_balance = '/column/Balance Total'

In [57]:
api_column_url_plomo = api_base_url_plomo + api_column_plomo
column_response_plomo = requests.get(api_column_url_plomo)

In [58]:
api_column_url_balance = api_base_url_plomo + api_column_balance
balance_plomo = requests.get(api_column_url_balance)

In [59]:
api_plomo = requests.get(api_base_url_plomo)
api_plomo.text

u'{"status":200,"success":true,"result":[{"Customer":"52 Avenue","Street1":"","Street2":"","City":"52 Roule","State":"DE","Zip":"","Primary Contact":"","Phone":"022-347-4918","Balance Total":"5,586.38"},{"Customer":"Ahmee","Street1":"206 W. 15th Street","Street2":"","City":"New York","State":"NY","Zip":"10011","Primary Contact":"","Phone":"","Balance Total":"0.00"},{"Customer":"Alexa Hakim","Street1":"","Street2":"","City":"","State":"","Zip":"","Primary Contact":"","Phone":"","Balance Total":"0.00"},{"Customer":"Amazon.com.kydc,Inc.","Street1":"1155 Worldwide Blvd.","Street2":"","City":"Hebron","State":"KY","Zip":"41048","Primary Contact":"","Phone":"","Balance Total":"913.75"},{"Customer":"American Express.","Street1":"","Street2":"","City":"","State":"","Zip":"","Primary Contact":"","Phone":"","Balance Total":"0.00"},{"Customer":"American Rag Cie, LLC","Street1":"160 S. La Brea Avenue","Street2":"","City":"Los Angeles","State":"CA","Zip":"90036","Primary Contact":"","Phone":"","Bala

In [ ]:
## See the instructor file for how to fill in the empty spaces

In [49]:
api_balance_plomo = balance_plomo.text
balance_plomo.text

u'{"status":200,"success":true,"result":["5,586.38","0.00","0.00","913.75","0.00","1,072.00","0.00","0.00","5,019.75","0.00","6,482.00","0.00","0.00","1,078.00","0.00","0.00","0.00","43,710.00","0.00","0.00","0.00","0.00","0.00","-2,000.00","0.00","0.00","0.00","833.00","5,157.00","0.00","0.00","0.00","0.00","11,658.00","1,080.00","-799.68","0.00","0.00","0.00","0.00","1,500.00","0.00","0.00","0.00","0.00","2,160.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","4,500.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","660.00","1,080.00","0.00","0.00","0.00","0.00","0.00","1,425.86","0.00","0.00","0.00","57,338.81","0.00","0.00","0.00","0.00","1,080.00","0.00","0.00","6,468.60","0.00","2,814.00","0.00","0.00","0.00","0.00","1,080.00","0.00","0.00","0.00","0.00","0.00","0.00","6,858.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","0.00","2,344.00","0.00","0.00","507.94","0.00","0.00","8,088.00","0.00","0.00","0.00"]}'

In [54]:
api_balance_plomo['result'].int(sum())

TypeError: string indices must be integers

In [25]:
plomo_dict = json.loads(api_plomo.text) 

In [26]:
plomo_df = pd.DataFrame(plomo_dict['result'])
plomo_df.head()

,Balance Total,City,Customer,Phone,Primary Contact,State,Street1,Street2,Zip
0,"5,586.38",52 Roule,52 Avenue,022-347-4918,,DE,,,
1,0.00,New York,Ahmee,,,NY,206 W. 15th Street,,10011
2,0.00,,Alexa Hakim,,,,,,
3,913.75,Hebron,"Amazon.com.kydc,Inc.",,,KY,1155 Worldwide Blvd.,,41048
4,0.00,,American Express.,,,,,,


In [41]:
plomo_df['api_column_url_balance'].sum()

TypeError: 'type' object has no attribute '__getitem__'

## Kimono Labs



Web scraping with the Kimono Labs API

https://www.kimonolabs.com/

Kimono labs has undergone a bit of a redesign in recent months and now encourages crawling for all the data you'd want ahead of time and extracting the batch in a series of calls. To create an api, follow the kimono labs walkthrough. To generate a large amount of data like we will here, a series of screenshot walkthroughs have been added.

After creating an API from an advanced search query as walked through in class, navigate to the Crawl Setup tab from the API management page in kimono.

In [ ]:
from IPython.display import Image
Image(filename='KL_Crawl_Setup.png') 

Change the crawl strategy to Generated URL List

In [ ]:
from IPython.display import Image
Image(filename='KL_Generated_URL_List.png') 

From here you can automatically generate a list of pages to extract data from, with the caveat that the pages you are navigating all have roughly the same structure. Here we are going to look for the top boxoffice_gross_us from 1990 to 2014. 

In [ ]:
from IPython.display import Image
Image(filename='KL_Parameter_range.png') 

In [ ]:
#After you let the data crawl the page, you can update the data here.

import json
import urllib

results = json.load(urllib.urlopen("https://www.kimonolabs.com/api/eb81bu78?apikey=4WrTjTsFDGmHCLnYrAXeSLIjJYcfrG7j"))
results

In [ ]:
your_key = "4WrTjTsFDGmHCLnYrAXeSLIjJYcfrG7j"
your_endpoint = "eb81bu78"
import json
import urllib
import pandas as pd

def getMovies(api_key=your_key,endpoint=your_endpoint):
    """
    Creates list of top 50 movies by gross box office
    sales for a with ratings and sales
    """
    
    movies, ratings, sales, years = [], [], [], []
    #Remember to replace this link with the link to your specifc API
    url = "https://www.kimonolabs.com/api/{}?".format(endpoint) + \
            "apikey={}".format(api_key)
    data = json.load(urllib.urlopen(url))
    
    # Iterate through json object to collect data
    for n in xrange(data['count']):
        n_title = data['results']['collection1'][n]['title']['text']
        n_rating = float(data['results']['collection1'][n]['rating'])
        n_sales = float(data['results']['collection1'][n]['sales'][1:-1])
        n_year = int(data['results']['collection1'][n]['url'][-4:])
        movies.append(n_title)
        ratings.append(n_rating)
        sales.append(n_sales)
        years.append(n_year)
    data = pd.DataFrame({'movie':movies,'rating':ratings,'sales_in_M':sales,'year':years})
    
    return data
movies = getMovies()
movies.head()

Note a few things: The function we wrote relies on extracting pieces of a string and converting them to numeric types. Here, the year can be extracted from the url used to generate the movies, so we use the slice `[-4:]` to identify that. Similarly, the gross cost comes out as $##M, so we know we never want the first or last value. Thus the slice `[1:-1]` is correct.

###Exercise 2.1
Build your own api via kimono at https://www.kimonolabs.com/. Create the api we created in class to call the data.

What were the average sales in 1995?
What was the average rating?

###Bonus
What were the average sales for the 90s? How does that differ from the 2000s?